<a href="https://colab.research.google.com/github/astromad/MyDeepLearningRepo/blob/master/ProductClassificationTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf Classification_cache_TF
!rm -rf results_TF
!rm -rf logs_TF

In [ ]:
!pip install --upgrade tf-nightly

In [ ]:
#!rm -rf Classification_cache


In [ ]:
#!pip install transformers

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/ColabData/Amazon.csv",
                encoding="ISO-8859-1", error_bad_lines=False)

data = df[['category', 'label_title', 'label_description']]
data.dropna(subset=['category'], inplace=True)
print(data.head(3))

In [ ]:
# Remove rows if category is null
data.dropna(subset=['category'], inplace=True)

In [ ]:
encode_dict={}
def encode_label(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

data['encoded_category'] = data['category'].apply(lambda x: encode_label(x))

In [ ]:
newData=pd.DataFrame()
newData['desc']=data['label_title'] +' '+ data['label_description'] 
newData['encoded_category']=data['encoded_category']

In [ ]:
# drop any rows with description is null
newData.dropna(subset=['desc'], inplace=True)
nan_rows = newData[newData.isnull().T.any()]
print(nan_rows)

In [ ]:
newData.loc[20,'desc']

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
# Preprocessing on description text data, remove stop words, remove spaces, lowercase
# note: we are not lemmatize as Bert will take care of it
newData['desc']=newData.desc.str.replace("[^\w\s]", "").str.lower()
#newData['desc']=newData.desc.str.replace('\d+', '')
#newData['desc']=newData['desc'].apply(lambda x: [item for item in x.split() if item not in stop])
newData['desc']=newData['desc'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [ ]:
newData.loc[20,'desc']

In [ ]:
from future.utils import iteritems
label2idx = {t: i for i, t in enumerate(encode_dict)}
idx2label = {v: k for k, v in iteritems(label2idx)}

In [ ]:
#newData.dropna(subset=['desc'], inplace=True)

In [ ]:
# nan_rows = newData[newData.isnull().T.any()]
# print(nan_rows)

In [ ]:
print(newData)

In [ ]:
ClassMax=newData['encoded_category'].max()
print(ClassMax)

In [ ]:
train_size = 0.8
train_dataset=newData.sample(frac=train_size,random_state=200)
test_dataset=newData.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(newData.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

In [ ]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    #BertTokenizer
)
model_args = dict()
model_args['model_name'] = 'bert-base-uncased' 
model_args['cache_dir'] = "Classification_cache_TF/"
model_args['do_basic_tokenize'] = False

config = AutoConfig.from_pretrained(
    model_args['model_name'],
    # num_labels=num_labels,
    # id2label=label_map,
    # label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args['cache_dir'],
    return_dict=True,
    num_labels=ClassMax+1
)

tokenizer = AutoTokenizer.from_pretrained(
    model_args['model_name'],
    cache_dir=model_args['cache_dir'],
    is_pretokenized=model_args['do_basic_tokenize'],
    do_basic_tokenize = model_args['do_basic_tokenize']
)

In [ ]:
import torch
import re
class TorchClassificationDataset(torch.utils.data.Dataset):
    def __init__(self,dataset,max_len):
        self.len = len(dataset)
        self.data = dataset
        self.max_len=max_len
    def __getitem__(self, idx):
        #item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        description = str(self.data.desc[idx])
        #description = " ".join(description.split())
        #print(description)
        description = description[:self.max_len]
        # description = re.sub('[^a-zA-Z0-9\n\.]', ' ', description)
        # description = " ".join(description.split())
        #description = re.sub('[^a-zA-Z0-9\n\.]', ' ', description)
        #print(description)
        inputs = tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            #pad_to_max_length=True,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        item ={}
        item['input_ids']=torch.tensor(inputs['input_ids'], dtype=torch.long)
        item['token_type_ids']=torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        item['attention_mask']=torch.tensor(inputs['attention_mask'], dtype=torch.long)
        item['labels'] = torch.tensor(self.data.encoded_category[idx], dtype=torch.long)
        return item

    def __len__(self):
        return self.len

In [ ]:
#[np.put(np.zeros(ClassMax+1),y,1) for y in test_dataset['encoded_category']]
def one_hot(i):
    a = np.zeros(ClassMax+1)
    a[i] = 1
    return a
#[one_hot(y) for y in test_dataset['encoded_category']]

In [ ]:
import numpy as np
def TFClassificationDataset(dataframe,MAX_LEN):
        # description = str(train_dataset.desc[idx])
        # #description = " ".join(description.split())
        # #print(description)
        # description = description[:self.max_len]
        # description = re.sub('[^a-zA-Z0-9\n\.]', ' ', description)
        # description = " ".join(description.split())
        description =dataframe['desc'].tolist()        
        #description = [re.sub('\W+',' ', x) for x in description]
        #print(description)
        inputs = tokenizer.batch_encode_plus(
            description,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        arr = np.zeros(10)
        np.put(arr, 5, 1)
        return inputs['input_ids'],inputs['attention_mask'],inputs['token_type_ids'],[[x] for x in dataframe['encoded_category']]
        #return inputs['input_ids'],inputs['attention_mask'],inputs['token_type_ids'],[one_hot(x) for x in dataframe['encoded_category']]

        # print(inputs['input_ids'])
        # print(tf.data.Dataset.from_tensor_slices(inputs['input_ids']))
        # return {
        #     tf.data.Dataset.from_tensor_slices(inputs['input_ids']),
        #     tf.data.Dataset.from_tensor_slices(inputs['attention_mask']),
        #     tf.data.Dataset.from_tensor_slices(inputs['token_type_ids']),
        #     tf.data.Dataset.from_tensor_slices([[x] for x in dataframe['encoded_category']])
        # }


In [ ]:
MAX_LEN=256
LEARNING_RATE = 3e-03

In [ ]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

In [ ]:
import tensorflow as tf
def createDataset(framework='pt'):
  if framework=='pt':
    train_ds = TorchClassificationDataset(train_dataset,MAX_LEN)
    test_ds= TorchClassificationDataset(test_dataset,MAX_LEN)
  else:
    input_ids_train,attention_masks_train,token_ids_train,label_ids_train= TFClassificationDataset(train_dataset,MAX_LEN)
    input_ids_test,attention_masks_test,token_ids_test,label_ids_test= TFClassificationDataset(test_dataset,MAX_LEN)

    # train_ds= TFClassificationDataset(train_dataset.loc[0:1],MAX_LEN)

    # print(input_ids_train)
    # print(attention_masks_train)
    # print(token_ids_train)
    # print(label_ids_train)
    train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features)
    test_ds=tf.data.Dataset.from_tensor_slices((input_ids_test,attention_masks_test,token_ids_test,label_ids_test)).map(example_to_features)
  return train_ds,test_ds

In [ ]:
train_ds,test_ds = createDataset('tf')
print('One record of Training dataset')
print(train_dataset.loc[1,'desc'])
print('----')
for x,y in train_ds.take(1):
  print(x)
  print(y)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import (
    TFAutoModelForSequenceClassification,
    #BertForSequenceClassification,
    TFTrainer,
    TFTrainingArguments
)

training_argsTF = TFTrainingArguments(
    output_dir='./results_TF',          
    num_train_epochs=20,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    warmup_steps=100,                
    weight_decay=0.01,               
    logging_dir='./logs_TF',            
    logging_steps=3,
    #debug=True
    #evaluate_during_training=True,
    #learning_rate=LEARNING_RATE
)
with training_argsTF.strategy.scope():
  modelTF = TFAutoModelForSequenceClassification.from_pretrained(
    model_args['model_name'],
    config=config,
    cache_dir=model_args['cache_dir'],
  )
trainerTF = TFTrainer(
    model=modelTF,                         
    args=training_argsTF,                  
    train_dataset=train_ds,        
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,  
)

In [ ]:
trainerTF.train() 

In [ ]:
trainerTF.evaluate()

In [ ]:
predictions, label_ids, metrics = trainerTF.predict(test_ds)
for key, value in metrics.items():
    print( key, value)

In [ ]:
inputs = tokenizer("Da-Lite Stand Master I - Cart for projector Projection Carts - Stand Master I Features The height of both the upper and lower shelves", return_tensors="tf")
print(inputs)
labels = tf.constant([76])
print(labels)

In [ ]:
import numpy as np
outputs = modelTF(inputs, labels=labels)
print(outputs.loss)
pred=np.argmax(outputs.logits)
print('prediction=',pred,idx2label[76])

In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# modelTF.compile(optimizer=optimizer, loss=loss)
# #n_steps = train_ds.shape[0] 
# modelTF.fit(train_ds,epochs=1, steps_per_epoch=256)